# Getting Started

In [2]:
from z3 import *

In [3]:
# to create an integer variable in Z3.
x = Int('x')
y = Int('y')

constraints = x > 2, y < 10, x + 2*y == 7

In [4]:
# the "solve" function solves a system of contraints.
solve(constraints)

[y = 0, x = 7]


In [5]:
x = Int('x')
y = Int('y')
print(simplify(x + y + 2*x + 3))
print(simplify(x < y + x + 2))

3 + 3*x + y
Not(y <= -2)


Z3 provides functions for traversing expressions.

In [10]:
x = Int('x')
y = Int('y')
n = x + y >= 3

print(f'num args: {n.num_args()}')
print(f'children: {n.children()}')
print(f'1st child: {n.arg(0)}')
print(f'2nd child: {n.arg(1)}')
print(f'operator: {n.decl()}')
print(f'operator name: {n.decl().name()}')

num args: 2
children: [x + y, 3]
1st child: x + y
2nd child: 3
operator: >=
operator name: >=


Z3 can solve nonlinear polynomial contraints.

In [14]:
# to create the real number variable.
x = Real('x')
y = Real('y')

constraints = x**2 + y**2 > 3, x**3 + y < 5
solve(constraints)

[x = 1/8, y = 2]


In [18]:
x = Real('x')
y = Real('y')

constraints = x**2 + y**2 == 3, x**3 == 2
set_option(precision=30)
print('Solving and displaying a result with 30 decimal places')
solve(constraints)

Solving and displaying a result with 30 decimal places
[x = 1.259921049894873164767210607278?,
 y = -1.188528059421316533710369365015?]


Rational number in Z3

In [25]:
print(1/3)
print(RealVal(1)/3)
print(Q(1,3))
print('------------------')
x = Real('x')
print(x + 1/3)
print(x + Q(1,3))
print(x + "1/3")
print(x + 0.25)

0.3333333333333333
1/3
1/3
------------------
x + 3333333333333333/10000000000000000
x + 1/3
x + 1/3
x + 1/4


Rational numbers can also be displayed in decimal notaion.

In [26]:
x = Real('x')

constraint = 3*x == 1
solve(constraint)

[x = 1/3]


In [28]:
set_option(rational_to_decimal=True)
solve(constraint)

[x = 0.333333333333333333333333333333?]


In [29]:
set_option(precision=10)
solve(constraint)

[x = 0.3333333333?]


A system of constraints may not have a solution.
In this case, we say the system is unsatisfiable.

In [30]:
x = Real('x')

constraints = x > 4, x < 0
solve(constraints)

no solution


In [32]:
x = Real('x')
constraint = x**2 + 2*x + 2 == 0
solve(constraint)

no solution


# Boolean Logic

Z3 supports Boolean operators:
And
Or
Not
Implies (implication)
If (if-then-else)
Bi-implications are represented using equality ==

In [34]:
p = Bool('p')
q = Bool('q')
r = Bool('r')

constraint1 = Implies(p, q)
constraint2 = r == Not(q)
constraint3 = Or(Not(p), r)

solve(constraint1, constraint2, constraint3)

[p = False, q = True, r = False]


The Python Boolean constants 'True' and 'False' can be used to build Z3 boolean expressions.

In [38]:
p = Bool('p')
q = Bool('q')
print(And(p, q, True))
print(simplify(And(p, q, True)))
print(simplify(And(p, False)))

And(p, q, True)
And(p, q)
False


A combination of polynomial and boolean constraints.

In [39]:
p = Bool('p')
x = Real('x')

constraints = Or(x < 5, x > 10), Or(p, x**2 == 2), Not(p)
solve(constraints)

[x = -1.4142135623?, p = False]


# Solvers

In [62]:
x = Int('x')
y = Int('y')

s = Solver()
print(s)

[]


In [63]:
# to add constraints.
s.add(x > 10, y == x + 2)
print(s)

[x > 10, y == x + 2]


In [64]:
print('Solving constraints in the solver s...')
# check() solves the asserted constraints.
# the result is 'sat'(satisfiable) if a solution was found.
# the result is 'unsat'(unsatisfiable) if no solution exists.
print(s.check())

Solving constraints in the solver s...
sat


In [65]:
print('Create a new scope')
s.push()
s.add(y < 11)
print(s)

Create a new scope
[x > 10, y == x + 2, y < 11]


In [66]:
print('Solving updated set of constraints...')
print(s.check())

Solving updated set of constraints...
unsat


In [67]:
print('Restoring state...')
s.pop()
print(s)

Restoring state...
[x > 10, y == x + 2]


In [68]:
print('Solving restored set of constraints...')
print(s.check())

Solving restored set of constraints...
sat


In [69]:
print(s.model())

[y = 13, x = 11]


The following example shows an example that Z3 cannot solve.
The solver return 'unknown' in this case.
Recall that Z3 can solve nonlinear polynomial constraints, but 2**x is not a polynomial.

In [70]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
print(s.check())

unknown


The following example shows how to traverse the constraints asserted into a solver, and how to collect performance statistics for the check method.

In [71]:
x = Real('x')
y = Real('y')

s = Solver()
s.add(x > 1, y > 1, Or(x + y > 3, x - y < 2))
print('asserted constraints...')
for c in s.assertions():
    print(c)

asserted constraints...
x > 1
y > 1
Or(x + y > 3, x - y < 2)


In [72]:
print(s.check())

sat


In [73]:
print('Statistics for the last check method...')
print(s.statistics())
# traversing statistics
for k, v in s.statistics():
    print(f'{k}: {v}')

Statistics for the last check method...
(:arith-make-feasible 3
 :arith-max-columns   4
 :arith-max-rows      2
 :arith-rows          4
 :arith-upper         4
 :decisions           2
 :final-checks        1
 :max-memory          5.95
 :memory              5.50
 :mk-bool-var         10
 :num-allocs          760732
 :num-checks          1
 :rlimit-count        9156)
mk bool var: 1
decisions: 2
final checks: 1
num checks: 1
mk bool var: 5
arith-upper: 4
arith-rows: 4
arith-make-feasible: 3
arith-max-columns: 4
arith-max-rows: 2
mk bool var: 1
mk bool var: 1
mk bool var: 1
mk bool var: 1
num allocs: 760743
rlimit count: 9156
max memory: 5.95
memory: 5.5


In [74]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print(s.check())

sat


In [76]:
m = s.model()
print(f'x = {m}')

x = [z = 0, y = 2, x = 1.5]


In [79]:
print('Traversing model...')
for d in m.decls():
    print(f'{d.name()} = {m[d]}')

Traversing model...
z = 0
y = 2
x = 1.5


# Arithmetic

In [82]:
x = Real('x')
y = Int('Y')

a, b, c = Reals('a b c')
s, r = Ints('s r')
print(x + y + 1 + (a + s))
# ToReal() casts an integer expression into a real expression.
print(ToReal(y) + c)

x + ToReal(Y) + 1 + a + ToReal(s)
ToReal(Y) + c


In [86]:
a, b, c = Ints('a b c')
d, e = Reals('d e')

constraints = a > b + 2, a == 2*c + 10, c + b <= 1000, d >= e           
solve(constraints)

[b = 0, c = 0, e = 0, d = 0, a = 10]


In [87]:
x, y = Reals('x y')
# to put expression in sum-of-monomials form
t = simplify((x + y)**3, som=True)
print(t)

x*x*x + 3*x*x*y + 3*x*y*y + y*y*y


In [88]:
# to use power operator
t = simplify(t, mul_to_power=True)
print(t)

x**3 + 3*x**2*y + 3*x*y**2 + y**3


In [89]:
help_simplify()

algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
arith_ineq_lhs (bool) rewrite inequalities so that right-hand-side is a constant. (default: false)
arith_lhs (bool) all monomials are moved to the left-hand-side, and the right-hand-side is just a constant. (default: false)
bit2bool (bool) try to convert bit-vector terms of size 1 into Boolean terms (default: true)
blast_distinct (bool) expand a distinct predicate into a quadratic number of disequalities (default: false)
blast_distinct_threshold (unsigned int) when blast_distinct is true, only distinct expressions with less than this number of arguments are blasted (default: 4294967295)
blast_eq_value (bool) blast (some) Bit-vector equalities into bits (default: false)
bv_extract_prop (bool) attempt to partially propagate extraction inwards (default: false)
bv_ineq_consistency_test_max (unsigned int) max size of conjunctions on which to perform consistency test bas

# Puzzles

Dog, Cat, and Mouse

Consider the following puzzle. Spend exactly 100 dollars and buy exactly 100 animals. Dogs cost 15 dollars, Cats costs 1 dollar, Mice cost 25 cents each. You have to buy at least one of each. How many of each should you buy?

In [93]:
# create 2 integer variables
dog, cat, mouse = Ints('dog cat mouse')
# at least one dog
c1 = dog >=1
# at least one cat
c2 = cat >= 1
# at least one mouse
c3 = mouse >= 1
# want to buy 100 animals
c4 = dog + cat + mouse == 100
# we have 10000 cents (100 dollars)
# dog costs 1500 cents
# cat costs 100 cents
# mouse costs 25 cents
c5 = 1500*dog + 100*cat + 25*mouse == 10000

solve(c1, c2, c3, c4, c5)

[cat = 41, mouse = 56, dog = 3]


Sodoku

In [102]:
# 9x9 matrix of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]

# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
             for j in range(9) ]

# each 3x3 square contains a digit at most once
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

# sudoku instance, we use '0' for empty cells
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

instance_c = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print("failed to solve")

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]
